In [ ]:
!pip install openai
!pip install langchain

In [ ]:
!pip install google-search-results
!pip install wikipedia
!pip install faiss-cpu 
!pip install sentence_transformers 
!pip install tiktoken 

In [ ]:
# LLM은 OpenAI 외에도 여러 종류

from langchain.llms import OpenAI

llm = OpenAI(model_name='gpt-3.5-turbo-instruct', temperature=0.9)
llm('1990년대 재즈 5곡만 추천해줘.')

In [ ]:
# 예전에 사용한대로 AI, Human, System message를 배열로 전달

from langchain.chat_models import ChatOpenAI
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

chat = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0.9)
sys = SystemMessage(content="당신은 음악 추천을 해주는 전문 AI입니다.")
msg = HumanMessage(content='1990년대 재즈 5곡만 추천해줘.')

aimsg = chat([sys, msg])
aimsg.content

In [ ]:
# 예전에 사용한대로 AI, Human, System message를 배열로 전달

from langchain.chat_models import ChatOpenAI
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

chat = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0.9)
sys = SystemMessage(content="당신은 여행을 추천해주는 전문 AI입니다.")
msg1 = HumanMessage(content='베트남 사파에 1박 2일 여행 가면 충분할까?')
msg2 = AIMessage(content='사파는 1박 2일로 가기에는 너무 멉니다. 일정을 조정하거나 다른 여행지를 선택하시면 어떨까요?')
msg3 = AIMessage(content='음 ... 그러면 어디가 좋을까?')

aimsg = chat([sys, msg1, msg2, msg3])
aimsg.content

In [ ]:
# PromptTemplate를 사용하여 각 LLM에 맞는 prompt를 지정할 수 있음

from langchain.prompts import PromptTemplate

prompt = PromptTemplate(
    input_variables=["상품"],
    template="{상품} 만드는 회사 이름 추천해줘. 기억에 남는 한글 이름으로",
)

prompt.format(상품="AI 여행 추천 서비스") # 변수가 여러 개 일 때, 지정

In [ ]:
from langchain.chains import LLMChain

chat = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0.9)
chain = LLMChain(llm=chat, prompt=prompt)

chain.run(상품="AI 여행 추천 서비스")

In [ ]:
# 각 Prompt 별 template 지정도 가능

from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

chat = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0)  

template="You are a helpful assisstant that translates {input_language} to {output_language}."
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template="{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])


chatchain = LLMChain(llm=chat, prompt=chat_prompt)
chatchain.run(input_language="English", output_language="Japanese", text="I love programming.")

In [ ]:
!pip install numexpr

In [ ]:
# ChatOpenAI를 LLM으로 사용하는 agent

from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType

chat = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0) 
tools = load_tools(["wikipedia", "llm-math"], llm=chat)

agent = initialize_agent(tools, llm=chat, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True) 

agent.run("아마존의 창업자는 누구인가? 그의 나이와 배우자의 나이를 곱하면 얼마인가?")

In [ ]:
agent.tools

In [ ]:
print(agent.tools[0].description)
print(agent.tools[1].description)

In [ ]:
from langchain import ConversationChain

chat = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0.9)
conversation = ConversationChain(llm=chat, verbose=True)
conversation.predict(input="인공지능에서 LLM이 뭐야?")

In [ ]:
conversation.predict(input="PLM과의 차이를 설명해줘.")

In [ ]:
conversation.memory

In [ ]:
# https://python.langchain.com/docs/modules/data_connection/document_loaders/
# https://api.python.langchain.com/en/latest/api_reference.html#module-langchain.document_loaders

from langchain.document_loaders import WebBaseLoader

loader = WebBaseLoader(web_path="https://ko.wikipedia.org/wiki/IU")
documents = loader.load()

from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)
len(docs)

In [ ]:
docs[:3]

In [ ]:
from langchain.chains.summarize import load_summarize_chain

chain = load_summarize_chain(chat, chain_type="map_reduce", verbose=True)
chain.run(docs[1:3])

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.embeddings import OpenAIEmbeddings

# embeddings = OpenAIEmbeddings()
embeddings = HuggingFaceEmbeddings() # sentence-transformers/all-mpnet-base-v2

from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import FAISS

index = VectorstoreIndexCreator(
    vectorstore_cls=FAISS,
    embedding=embeddings,
    ).from_loaders([loader])

# 파일로 저장
index.vectorstore.save_local("faiss-nj-IU")
# db = FAISS.load_local("faiss-nj-IU", embeddings=embeddings)

In [ ]:
index.query("아이유의 데뷔곡은?", llm=chat, verbose=True)

In [ ]:
index.query("아이유의 나이는?", llm=chat, verbose=True)

In [ ]:
index.query("아이유의 나이는? (오늘은 2024년 3월 12일)", llm=chat, verbose=True)

In [ ]:
index.query("아이유는 지금까지 총 콘서트를 몇회했는가?", llm=chat, verbose=True)